### **발표 양식(실제 DB)**

`-` DB의 스키마 표시 `SELECT * FROM sqlite_schema WHERE name = 'table_name'`

`-` 기본키로 설정을 해놨는데 삽입(중복된 것)을 하면 어떻게 되는지 `INSERT`구문

`-` 수정, 삽입, 갱신에 대한 구문도 간단히 해야 한다. `INSERT, DELETE`구문

`-` 조인을 통한 데이터 결합 `SELECT FULL JOIN`구문

`-` 시간이 좀 더 있다면 어떤 데이터를 입력이나 수정할 지 현장에서 요구할 수도 있다.

`-` 간단한 분석

`-` 등등...

    cf) 발표와 질의응답 : 발표를 누가 하든지 질의응답에서 대답을 잘 못하면 개별 감점이 있습니다.

    보강이 필요한데, 영상으로 진행할 예정입니다. 세 시간 정도 앞에서 배운 내용을 요약해주는 형식입니다. 왠만해서는 영상으로 보강이 가능하도록 할 겁니다.

    퀴즈는 발표주차 바로 전에 보도록 하겠습니다. 15~20분 정도 분량입니다. 간단히 확인할 수 있는 정도의 문제.

## 1. SQLite에 대한 설명

오픈 소스, 가벼운 유형으로 모든 시스템에서 사용이 가능하다.

데이터 형식의 다양성(스키마 지정), 수평 확장성(데이터베이스 네트워크 간 확장), 데이터 관리의 엄격성에서는 다소 약점이 있고 이런 부분에서는 MySQL이 강점을 가지고 있다.

> 실제 상황과의 괴리 : 실제는 서버에서 돌아가고, 대용량 데이터가 여러 데이터베이스간 교차하여 작동함.

지원하는 데이터 유형

* TEXT 문자열(UTF-8, UTF-16BE, UTF16LE)
* REAL 8바이트 부동 소수점 값. float32
* NULL
* INTEGER 8바이트 정수값. int32
* BLOB 입력형태 그대로 저장(이미지, 오디오, 멀티미디어 파일)

## 2. Imports

In [2]:
# install.packages("DBI")
# install.packages("RSQLite")
library(DBI)
library(RSQLite)

## 3. Function

### **A. Connecting**

---

In [92]:
data("mtcars")
mtcars$car_names = rownames(mtcars) ## 인덱스를 데이터로 따로 받아 새 열로 추가
rownames(mtcars) = NULL  ## 인덱스 초기화
head(mtcars)

,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb,car_names
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,21.0,6,160,110,3.90,2.620,16.46,0,1,4,4,Mazda RX4
2,21.0,6,160,110,3.90,2.875,17.02,0,1,4,4,Mazda RX4 Wag
3,22.8,4,108,93,3.85,2.320,18.61,1,1,4,1,Datsun 710
4,21.4,6,258,110,3.08,3.215,19.44,1,0,3,1,Hornet 4 Drive
5,18.7,8,360,175,3.15,3.440,17.02,0,0,3,2,Hornet Sportabout
6,18.1,6,225,105,2.76,3.460,20.22,1,0,3,1,Valiant


`-` Connecting Database File : `dbConnect()`

In [93]:
conn = dbConnect(RSQLite::SQLite(), "CarsDB.db")  ## 데이터베이스 파일이 존재하지 않으면 생성

`-` Writing table : `dbWriteTable()` 정제된 데이터프레임 삽입에 유리(append)

In [94]:
dbWriteTable(conn, "cars_data", mtcars)  ## Write table(relation)
dbListTables(conn)  ## debuging

[1] "cars_data"

In [95]:
car <- c('Camaro', 'California', 'Mustang', 'Explorer')
make <- c('Chevrolet','Ferrari','Ford','Ford')
df1 <- data.frame(car,make)
car <- c('Corolla', 'Lancer', 'Sportage', 'XE')
make <- c('Toyota','Mitsubishi','Kia','Jaguar')
df2 <- data.frame(car,make)

In [96]:
dfList = list(df1, df2)

for (i in 1:length(dfList)) {
    dbWriteTable(conn, "Cars_and_Makes", dfList[[i]], append = TRUE)  ## write new table
}

dbListTables(conn)

[1] "Cars_and_Makes" "cars_data"

* arguments

> append = TRUE : 기존 테이블에 추가. 해당 기능 덕분에 스키마를 만들어두고 그대로 집어넣기가 용이하다.
>
> 작성 후 반드시 dbDisconnect()를 하라고 경고가 발생

### **B. Query**

---

`-` Query : `dbGetQuery()` 작업은 불가능하고 쿼리 결과 테이블만 반환할 수 있음.

In [98]:
# dbGetQuery(conn, "SELECT * FROM Cars_and_Makes")
# dbGetQuery(conn, "SELECT * FROM cars_data LIMIT 20")
dbGetQuery(conn, "SELECT car_names, hp, cyl FROM cars_data")

car_names,hp,cyl
<chr>,<dbl>,<dbl>
Mazda RX4,110,6
Mazda RX4 Wag,110,6
Datsun 710,93,4
Hornet 4 Drive,110,6
Hornet Sportabout,175,8
Valiant,105,6
Duster 360,245,8
Merc 240D,62,4
Merc 230,95,4


`-` Query : `dbExecute()` 작업만 수행하고 결과물을 반환하지 않음. 스키마 생성 시 유용, 스키마 생성 쿼리문은 하단 기술

In [3]:
conn = dbConnect(SQLite(), ".temp.db")

In [100]:
## creating table with schema
dbExecute(conn, "CREATE TABLE mytable
          (a INTEGER PRIMARY KEY, 
         b TEXT)")

[1] 0

In [101]:
dbExecute(conn, "INSERT INTO mytable VALUES(1, 'test'), (2, 'test'), (6, 'test')")

[1] 3

In [103]:
dbGetQuery(conn, "SELECT * FROM mytable")

a,b
<int>,<chr>
1,test
2,test
6,test


`-` Query : `dbSendQuery()` 위 함수와 비슷하나, 작업을 수행하고 어떤 데이터가 들어갔는지 결과를 반환

In [105]:
dbSendQuery(conn, "INSERT INTO mytable VALUES(4, 'test')")

<SQLiteResult>
  SQL  INSERT INTO mytable VALUES(4, 'test')
  ROWS Fetched: 0 [complete]
       Changed: 1

In [106]:
dbGetQuery(conn, "SELECT * FROM mytable")

Warning message:
“Closing open result set, pending rows”


a,b
<int>,<chr>
1,test
2,test
4,test
6,test


`-` Removing table : `dbRemoveTable()` 기존 테이블을 제거

In [ ]:
dbRemoveTable(conn, 'test_table')

## 4. SQLite Query

`-` `SELECT`

* 사용예시 : `"SELECT col_name1 AS col1, AVG(col2) AS average_col_2 FROM table GROUP BY col1 ORDER BY average_col_2"`

> 범용적인 문법, 테이블을 바꾸지 않음. `dbGetQuery()`에서 주로 사용.
>
> `AS`로 열 이름을 변경해서 호출, `AVG()`로 평균을 호출, `GROUP BY`에 해당하는 열 그룹을 기준으로 그룹화하여 통계량 산출, `ORDER BY`에 해당하는 열 그룹을 기준으로 sorting.

`-` `INSERT`

`"INSERT INTO table (col1, col2, ...) VALUES (val11, val12, ...), (val21, val22, ...), ..."`

> 테이블에 튜플 삽입, `dbExecute()`, `dbSendQuery()`에서 사용.

`-` `DELETE`

`"DELETE FROM table WHERE col_name = value"`

> 테이블의 튜플 제거, `dbExecute()`, `dbSendQuery()`에서 사용.

`-` `UPDATE`

`"UPDATE table SET col_name = value WHERE col_name = value"`

> 테이블의 튜플 수정, `dbExecute()`, `dbSendQuery()`에서 사용.

`-` `PRAGMA`

`"PRAGMA foreign_keys = ON"`

> `PRAGMA`를 통해 데이터베이스 동작 방식을 선언할 수 있다. 위 경우 파라메터를 변경하였다. `dbExecute()`, `dbSendQuery()`에서 사용.

`-` **스키마 지정**

In [ ]:
"CREATE TABLE table_name 
(x1 INTEGER PRIMARY KEY,
 x2 TEXT NOT NULL,
 x3 DEFAULT CURRENT_TIMESTAMP,
 x4 INTEGER CHECK (x4 >= 100),
 id INTEGER UNIQUE,
 FOREIGN KEY (y1) REFERENCES ref_table (y1)
 ON UPDATE SET CASCADE
 ON DELETE SET NULL)"

> 테이블을 생성하고, 스키마를 지정.

`INTEGER`, `TEXT`, `REAL`, `DATE`, $\cdots$ : 자료형

`NOT NULL` : `NULL`값을 허용하지 않음

`UNIQUE` : 중복되는 값을 입력할 수 없음

`CHECK (logical)` : 값이 가질 수 있는 제약조건을 명시할 수 있음

`PRIMARY KEY` : 기본키, `UNIQUE`와 `NOT NULL`제약조건을 기본적으로 포함. 사용 전 반드시 DB 파일의 외래키 사용 활성화 필요. (`dbExecute(conn, "PRAGMA foreign_keys = ON")`)

`FOREIGN KEY (att1) REFERENCES table (att2)` : 외래키, 어느 릴레이션의 어느 속성을 참조할 것인지 `REFFERENCES`에서 명시하고`(att2)`, 어느 이름으로 가져올 것인지 명시`(att1)`

외래키의 경우 기존 테이블의 정보가 변경되거나 삭제되었을 때 어떻게 변할 것인지 명시해야 함.

> `ON UPDATE SET NULL` : 기본키 변화 시 외래키는 `NULL`
>
> `ON DELETE SET NULL` : 기본키 제거 시 외래키는 `NULL`
> 
> `ON UPDATE SET CASCADE` : 기본키 변화 시 외래키도 같이 변화
>
> `ON DELETE SET CASCADE` : 기본키 제거 시 외래키도 제거

기본키를 여러 개 지정하려면 속성을 스키마로 미리 넣어두고 `PRIMARY KEY (x1, x2)`로 하면 된다. 외래키를 기본키로 지정하려면 `FOREIGN KEY (y1) REFERENCES ref_table1 (y1), FOREIGN KEY (y2) REFERENCES ref_table2 (y2)`를 하고, `PRIMARY KEY (y1, y2)`를 하면 된다.

`-` `Join` 문법 (`demand_groups`, `supplier_groups`, `custom_groups` 릴레이션이 존재하는 상태)

In [ ]:
## Generating
dbExecute(conn, "CREATE TABLE demand_groups (
          group_id INTEGER PRIMARY KEY CHECK(GROUP_ID <= 100),
)")

In [ ]:
## Join
dbGetQuery(conn, "SELECT dgroup_name, idy,
           demand_groups.group_id AS d_id,
           supplier_groups.group_id AS s_id,
           FROM demand_groups
           INNER JOIN supplier_groups ON
           supplier_groups.group_id = demand_groups.group_id")

> `demand_groups`와 `supplier_groups`를 조인하는데, 기준이 되는 열은 `group_id`이다. 어느 릴레이션에서 속성을 가져올지는 `table_name.column_name`으로 입력한다. 판다스 데이터프레임을 다루는 방식과 유사하다. 어떤 테이블에서 가져올지를 명시하지 않으면 첫번째 걸로 된다.

조인은 FULL OUTER, RIGHT OUTER, LETF OUTER 다 됨.

In [ ]:
dbGetQuery(conn, "SELECT dgroup_name, idy,
           demand_groups.group_id AS d_id,
           supplier_groups.group_id AS s_id,
           FROM demand_groups
           FULL OUTER JOIN supplier_groups ON
           supplier_groups.group_id = demand_groups.group_id")

JOIN을 할 때, 알아서 열들을 다 가져온다고 생각하겠으나, 실제로는 SELECT를 통해 열들을 일일히 다 가져와줘야 한다.

In [ ]:
dbGetQuery(conn, "SELECT dgroup_name, idy,
           demand_groups.group_id AS d_id,
           supplier_groups.group_id AS s_id,
           custom_groups.group_id AS c_id,
           supplier_groups.idx AS s_idx,
           custom_groups.idz AS c_qty
           FROM demand_groups
           LEFT OUTER JOIN suppliers ON
           supplier_groups.group_id = demand_groups.group_id
           LEFT OUTER JOIN custom_groups ON
           custom_groups.group_id = demand_groups.group_id")

3개 이상의 테이블을 조인하는 것도 가능함. 가능하면 INNER, LEFT OUTER, FULL OUTER로만 사용하는 것을 추천.

In [ ]:
dbGetQuery(conn, "SELECT dgroup_name, idy,
           demand_groups.group_id AS d_id,
           supplier_groups.group_id AS s_id,
           /* custom_groups.group_id AS c_id, */
           supplier_groups.idx AS s_idx,
           /* custom_groups.idz AS c_qty */
           FROM demand_groups
           LEFT OUTER JOIN suppliers ON
           supplier_groups.group_id = demand_groups.group_id AND
           demand_groups.idy > supplier_groups.idx")

THETA JOIN은 그냥 부등호를 사용하면 된다.

In [ ]:
dbGetQuery(conn, "SELECT *
           FROM demand_groups
           CROSS JOIN supplier_groups")

카티션 프로덕트의 경우 그냥 CROSS 조인의 구문 유형으로 처리하고 있음.

In [ ]:
dbGetQuery(conn, "SELECT * FROM sqlite_schema
           WHERE name = 'gg_data'")

테이블의 스키마를 호출

In [8]:
conn = dbConnect(SQLite(), "tmp.db")
dbSendQuery(conn, "CREATE TABLE IF NOT EXISTS tmp (id INTEGER, name TEXT, address TRXT)")
dbSendQuery(conn, "ALTER TABLE tmp RENAME TO tmp1")  ## mv와 비슷하다고 보면 됨. 이름을 바꿈.
dbGetQuery(conn, "SELECT * FROM tmp1")

<SQLiteResult>
  SQL  CREATE TABLE IF NOT EXISTS tmp (id INTEGER, name TEXT, address TRXT)
  ROWS Fetched: 0 [complete]
       Changed: 0

Warning message:
“Closing open result set, pending rows”


<SQLiteResult>
  SQL  ALTER TABLE tmp RENAME TO tmp1
  ROWS Fetched: 0 [complete]
       Changed: 0

Warning message:
“Closing open result set, pending rows”


id,name,address
<int>,<chr>,<dbl>


In [15]:
dbSendQuery(conn, "CREATE TABLE IF NOT EXISTS tmp2 (id INTEGER, name TEXT)")
dbSendQuery(conn, "ALTER TABLE tmp2 ADD COLUMN misc TEXT DEFAULT(NULL)")
dbGetQuery(conn, "SELECT * FROM tmp2")

<SQLiteResult>
  SQL  CREATE TABLE IF NOT EXISTS tmp2 (id INTEGER, name TEXT)
  ROWS Fetched: 0 [complete]
       Changed: 0

Warning message:
“Closing open result set, pending rows”


<SQLiteResult>
  SQL  ALTER TABLE tmp2 ADD COLUMN misc TEXT DEFAULT(NULL)
  ROWS Fetched: 0 [complete]
       Changed: 0

Warning message:
“Closing open result set, pending rows”


id,name,misc
<int>,<chr>,<chr>


In [16]:
dbGetQuery(conn, "SELECT * FROM sqlite_schema")

type,name,tbl_name,rootpage,sql
<chr>,<chr>,<chr>,<int>,<chr>
table,cars_data,cars_data,2,"CREATE TABLE `cars_data` ( `mpg` REAL, `cyl` REAL, `disp` REAL, `hp` REAL, `drat` REAL, `wt` REAL, `qsec` REAL, `vs` REAL, `am` REAL, `gear` REAL, `carb` REAL, `car_names` TEXT )"
table,Cars_and_Makes,Cars_and_Makes,3,"CREATE TABLE `Cars_and_Makes` ( `car` TEXT, `make` TEXT )"
table,tmp2,tmp2,4,"CREATE TABLE tmp2 (id INTEGER, name TEXT, misc TEXT DEFAULT(NULL))"


ALTER를 통한 작업

`-` 테이블 변경으로 인한 스키마 변경(잡기술)

In [ ]:
dbSendQuery(conn, "CREATE TABLE IF NOT EXISTS tmpp (id INTEGER, name TEXT)")
dbExecute(conn, "INSERT INTO tmpp (id, name) SELECT id, name FROM tmp1")  ## 새로운 테이블에 속성 붙임.
dbGetQuery(conn, "SELECT * FROM tmpp")
dbExecute(conn, "DROP TABLE tmp1")
dbExecute(conn, "ALTER TABLE tmpp RENAME TO tmp1")

> 처음부터 스키마를 잘 구성하는 편이 좋다. 스키마 깨지면 그냥 처음부터 다시 구성하는 게 좋다.

`-` 통계량 `COUNT`

In [6]:
conn = dbConnect(SQLite(), "./R/CarsDB.db")

In [7]:
dbListTables(conn)

[1] "Cars_and_Makes" "cars_data"

In [5]:
data("mtcats")
mtcars$car_names = rownames(mtcars)
rownames(mtcars) = NULL
head(mtcars)

Warning message in data("mtcats"):
“data set ‘mtcats’ not found”


,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb,car_names
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,21.0,6,160,110,3.90,2.620,16.46,0,1,4,4,Mazda RX4
2,21.0,6,160,110,3.90,2.875,17.02,0,1,4,4,Mazda RX4 Wag
3,22.8,4,108,93,3.85,2.320,18.61,1,1,4,1,Datsun 710
4,21.4,6,258,110,3.08,3.215,19.44,1,0,3,1,Hornet 4 Drive
5,18.7,8,360,175,3.15,3.440,17.02,0,0,3,2,Hornet Sportabout
6,18.1,6,225,105,2.76,3.460,20.22,1,0,3,1,Valiant


In [8]:
dbGetQuery(conn, "SELECT COUNT(cyl) FROM cars_data")
dbGetQuery(conn, "SELECT COUNT(DISTINCT mpg) FROM cars_data")  ## mpg에서 구별되는(distincted) 값 개수

COUNT(cyl)
<int>
32


COUNT(DISTINCT mpg)
<int>
25


In [18]:
length(mtcars$cyl)
length(unique(mtcars$mpg))

[1] 32

[1] 25

In [9]:
dbGetQuery(conn, "SELECT cyl, COUNT(*) FROM cars_data
           GROUP BY cyl HAVING COUNT(*) > 0
           ORDER BY COUNT(*)")

cyl,COUNT(*)
<dbl>,<int>
6,7
4,11
8,14


속성 cyl을 기준으로 튜플의 개수(COUNT)를 반환. 개수가 0보다 큰 것을 cyl별 개수(COUNT)로 정렬(ORDER BY)하여 반환

중앙값 통계량 산출

짝수 : 가운데 두 개의 값의 평균(floor(n/2), floor(n/2 + 1))

홀수 : 가운데 값(n/2)

In [14]:
dbGetQuery(conn, "SELECT cyl FROM cars_data
           ORDER BY cyl
           LIMIT 2 - (SELECT COUNT(*) FROM cars_data) % 2
           OFFSET (SELECT (COUNT(*) - 1) / 2 FROM cars_data)")

cyl
<dbl>
4
6


짝수면 두 줄을 봐야 하고(`LIMIT 2`), 홀수면 한 줄을 봐야 한다(`LIMIT 2 - n%2`)

(n-1)/2번째 부터 두 줄(한 줄)을 봐야 한다(`OFFSET`)